In [1]:
track_progress = True
# If true, the model prints statements as it completes certain long calculations

# Run necessary background code and generate baseline

In [9]:
# Background functions and calculations of adjustment factors
execfile('import_packages.py')
execfile('read_in_data.py')
execfile('amt_model.py')
execfile('ftc_model.py')
execfile('sec199_model.py')
execfile('ccr_model.py')
#execfile('interest_model.py')
execfile('usercode_taxcalc.py')

All packages successfully imported
Data imports complete
Beginning CCR model
Corporate depreciation adjustment calculated
Noncorporate depreciation adjustment calculated


In [10]:
execfile('gen_baseline.py')

Taxable income calculated
Investment matrices constructed
Corporate depreciation calculated
Noncorporate depreciation calculated
Capital paths calculated
Corporate net interest deduction calculated
Noncorporate interest deduction calculated
You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.
MTR calculations complete for 2017
MTR calculations complete for 2018
MTR calculations complete for 2019
MTR calculations complete for 2020
MTR calculations complete for 2021
MTR calculations complete for 2022
MTR calculations complete for 2023
MTR calculations complete for 2024
MTR calculations complete for 2025
MTR calculations complete for 2026
MTR calculations complete for 2027
Marginal tax rates calculated
Baseline complete


# Specify tax reform

In [45]:
btax_dict1 = {
    2017: {
        #'tau_c': 0.357,
        #'tau_nc': 0.0,
        #'tau_amt': 0.2,
        #'depr_3yr_method': 'GDS',
        #'depr_3yr_bonus': 0.5,
        #'depr_5yr_method': 'GDS',
        #'depr_5yr_bonus': 0.5,
        #'depr_7yr_method': 'GDS',
        #'depr_7yr_bonus': 0.5,
        #'depr_10yr_method': 'GDS',
        #'depr_10yr_bonus': 0.5,
        #'depr_15yr_method': 'GDS',
        #'depr_15yr_bonus': 0.5,
        #'depr_20yr_method': 'GDS',
        #'depr_20yr_bonus': 0.5,
        #'depr_25yr_method': 'GDS',
        #'depr_25yr_bonus': 0.0,
        #'depr_275yr_method': 'GDS',
        #'depr_275yr_bonus': 0.0,
        #'depr_39yr_method': 'GDS',
        #'depr_39yr_bonus': 0.0,
        #'depr_land_bonus': 0.0,
        #'inventory_method': 'Mix'
    }
}
btax_dict2 = {
    'undepBasis_corp_hc': {0: 0.0},
    'undepBasis_noncorp_hc': {0: 0.0},
    'amt_repeal': {9e99: False},
    'pymtc_repeal': {9e99: False},
    'ftc_hc': {9e99: 0.0},
    'sec199_hc': {9e99: 0.0},
    'oldIntPaid_corp_hc': {0: 0.0},
    'newIntPaid_corp_hc': {0: 0.0},
    'netIntPaid_corp_hc': {0: 0.0},
    'oldIntPaid_noncorp_hc': {0: 0.0},
    'newIntPaid_noncorp_hc': {0: 0.0},
    'reclassify_taxdep_gdslife': {2018: {39: 25}},
    'reclassify_taxdep_adslife': {9e99: {}}
}
iit_params_ref = {}
execfile('reform_implementation.py')

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.
MTR calculations complete for 2017
MTR calculations complete for 2018
MTR calculations complete for 2019
MTR calculations complete for 2020
MTR calculations complete for 2021
MTR calculations complete for 2022
MTR calculations complete for 2023
MTR calculations complete for 2024
MTR calculations complete for 2025
MTR calculations complete for 2026
MTR calculations complete for 2027
All parameters updated


# Calculate the investment response

In [68]:
# Set all elasticities
#     inv_usercost_c: elasticity of corporate investment w.r.t. the user cost of capital
#     inv_usercost_nc: elasticity of noncorporate investment w.r.t. the user cost of capital
#     inv_eatr: semi-elasticity of investment w.r.t. the effective average tax rate
#     debt_taxshield_c: semi-elasticity of the corporate debt/asset ratio w.r.t. the marginal tax shield, tau * (1 - hc)
#     debt_taxshield_nc: semi-elasticity of the noncorporate debt/asset ratio w.r.t. the marginal tax shield, tau * (1 - hc)
#     legalform_ratediff: semi-elasticity of the corporate share of business activity w.r.t. the (Tc - Tnc) tax rate differential


elast_dict = {
    'inv_usercost_c': -1.0,
    'inv_usercost_nc': -1.0,
    'inv_eatr_c': 0.0,
    'inv_eatr_nc': 0.0,
    'mne_share_c': 0.0,
    'mne_share_nc': 0.0,
    'debt_taxshield_c': 0.0,
    'debt_taxshield_nc': 0.0,
    'legalform_ratediff': 0.0
}
execfile('responses.py')

In [69]:
execfile('mini_combined.py')
response_results = inv_response(2017)

In [70]:
response_results

,Asset,deltaIc2014,deltaInc2014,MPKc2014,MPKnc2014,deltaIc2015,deltaInc2015,MPKc2015,MPKnc2015,deltaIc2016,...,MPKc2025,MPKnc2025,deltaIc2026,deltaInc2026,MPKc2026,MPKnc2026,deltaIc2027,deltaInc2027,MPKc2027,MPKnc2027
0,Aerospace products and parts manufacturing,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.300471,0.298005,0.000000,0.000000,0.300570,0.297631,0.000000,0.000000,0.300570,0.299022
1,Air transportation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.135820,0.124395,0.050373,0.040431,0.135979,0.124524,0.050373,0.040394,0.135979,0.124512
2,Aircraft,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.179338,0.173513,0.000000,0.000000,0.179463,0.173736,0.000000,0.000000,0.179463,0.173830
3,"All other nonmanufacturing, n.e.c.",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.240471,0.238125,0.000000,0.000000,0.240570,0.237895,0.000000,0.000000,0.240570,0.239022
4,Amusement and recreation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.121377,0.116164,0.000000,0.000000,0.121502,0.116349,0.000000,0.000000,0.121502,0.116397
5,Autos,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.447847,0.436086,0.000000,0.000000,0.447996,0.436186,0.000000,0.000000,0.447996,0.436182
6,Books,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.236590,0.224179,0.000000,0.000000,0.236754,0.224283,0.000000,0.000000,0.236754,0.224252
7,"Chemical manufacturing, ex. pharma and med",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.240471,0.238125,0.000000,0.000000,0.240570,0.237895,0.000000,0.000000,0.240570,0.239022
8,Communication,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.124331,0.116279,0.000000,0.000000,0.124475,0.116424,0.000000,0.000000,0.124475,0.116429
9,Communications,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.213685,0.205741,0.000000,0.000000,0.213821,0.205926,0.000000,0.000000,0.213821,0.205976


In [71]:
legal_response(2017)
print rescale_corp
print rescale_noncorp

[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]


# Run other responses and recalculate corporate tax revenue

In [72]:
execfile('gen_reform.py')

New investment matrices constructed
New corporate depreciation calculated
New noncorporate depreciation calculated
New capital paths calculated
New earnings calculated
New corporate net interest deduction calculated
New noncorporate interest deduction calculated
New taxable income calculated
Reform corporate tax revenue complete


### Check capital results

In [73]:
capPath_ref_corp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,1027.370558,6192.813634,1105.741653,8525.386334,477.426504,884.575577,2014
1,985.082931,6762.771552,1092.837295,9180.871516,546.326287,905.241147,2015
2,887.419889,7287.566860,915.439816,9843.598272,600.386409,830.188301,2016
3,920.096877,7703.955635,949.148567,10332.136132,636.418240,866.909916,2017
4,966.506508,8141.813810,996.773205,10850.336631,672.589389,825.113557,2018
5,1001.514799,8603.922611,1032.877800,11397.318140,709.400630,805.821897,2019
6,1036.052277,9076.317723,1068.496839,11958.320861,746.020014,643.427528,2020
7,1074.940869,9557.093760,1108.607509,12530.893582,782.442991,727.467753,2021
8,1117.258729,10047.626952,1152.255919,13115.561359,819.220000,799.627580,2022
9,1161.520783,10552.652176,1197.907648,13717.664353,856.961682,861.095411,2023


In [74]:
capPath_base_corp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,1027.370558,6192.813634,1105.741653,8525.386334,477.426504,884.575577,2014
1,985.082931,6762.771552,1092.837295,9180.871516,546.326287,905.241147,2015
2,887.419889,7287.566860,915.439816,9843.598272,600.386409,830.188301,2016
3,920.096877,7703.955635,949.148567,10332.136132,636.418240,866.909916,2017
4,958.577389,8141.813810,988.844086,10850.336631,672.589389,820.971756,2018
5,993.298476,8595.835401,1024.661477,11389.230930,709.190326,799.265383,2019
6,1027.552612,9059.898359,1059.997174,11941.901497,745.593171,635.671702,2020
7,1066.257070,9532.102796,1099.923709,12505.902618,781.793520,716.449543,2021
8,1108.396947,10013.937655,1143.394137,13081.872062,818.344742,785.544298,2022
9,1152.409391,10510.142544,1188.796255,13675.154721,855.857605,843.980733,2023


In [75]:
capPath_ref_noncorp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,271.318833,2566.871440,306.337312,6737.647577,159.355609,301.392835,2014
1,287.158825,2686.785985,338.014040,6905.064249,170.218220,328.459158,2015
2,292.681264,2837.223493,306.828509,7157.361498,182.182362,318.973624,2016
3,303.458510,2998.062147,318.126691,7406.366186,193.062471,336.049883,2017
4,319.379869,3168.458510,334.661506,7676.804170,204.035274,324.723608,2018
5,330.946293,3349.275602,346.781454,7963.095023,215.251805,320.001603,2019
6,342.357179,3535.188783,358.738419,8258.664750,226.448408,264.189094,2020
7,355.201388,3725.451398,372.199655,8561.834813,237.620684,295.371012,2021
8,369.175643,3920.299881,386.845703,8871.263371,248.922170,322.475732,2022
9,383.793153,4121.393002,402.164860,9189.434369,260.530719,345.875481,2023


In [76]:
capPath_base_noncorp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,271.318833,2566.871440,306.337312,6737.647577,159.355609,301.392835,2014
1,287.158825,2686.785985,338.014040,6905.064249,170.218220,328.459158,2015
2,292.681264,2837.223493,306.828509,7157.361498,182.182362,318.973624,2016
3,303.458510,2998.062147,318.126691,7406.366186,193.062471,336.049883,2017
4,316.149825,3168.458510,331.431462,7676.804170,204.035274,322.837096,2018
5,327.601237,3345.981157,343.436398,7959.800578,215.169547,317.130245,2019
6,338.898645,3528.498656,355.279885,8251.974623,226.281416,260.912143,2020
7,351.663819,3715.266454,368.662085,8551.649869,237.366536,290.716170,2021
8,365.562034,3906.560722,383.232094,8857.524212,248.579435,316.542453,2022
9,380.077843,4104.042649,398.449550,9172.084016,260.098029,338.688164,2023


### Check combined results

In [77]:
combined_ref

,year,ebitda,taxDep,nid,sec199,taxinc,gbc_adj,tau,taxbc,ftc,amt,pymtc,taxrev
0,2014,2445.993351,884.575577,231.441998,34.306208,1295.669568,0.021643,0.347,421.555664,101.582376,4.457941,3.700228,320.731000
1,2015,2520.645171,905.241147,243.981131,34.855388,1336.567506,0.021643,0.347,434.862110,91.794692,4.529305,3.799722,343.797000
2,2016,2327.856171,830.188301,248.001094,34.816996,1214.849781,0.021643,0.347,395.260349,96.318143,4.524316,3.895522,299.571000
3,2017,2428.365149,866.909916,268.261581,34.952204,1258.241448,0.021643,0.347,409.378148,99.935946,4.541886,3.978088,310.006000
4,2018,2443.823519,825.113557,295.578104,35.018974,1288.112885,0.021643,0.347,419.097041,96.600049,4.550562,4.052119,322.995435
5,2019,2521.224568,805.821897,329.182016,35.349483,1350.871171,0.021643,0.347,439.515913,97.986091,4.593510,4.117569,342.005764
6,2020,2521.805512,643.427528,362.967959,35.856932,1479.553093,0.021643,0.347,481.383527,103.553749,4.659451,4.180064,378.309165
7,2021,2659.502559,727.467753,394.505947,36.599743,1500.929116,0.021643,0.347,488.338374,109.003947,4.755976,4.243011,379.847392
8,2022,2798.344249,799.627580,423.625155,37.604625,1537.486888,0.021643,0.347,500.232715,114.806528,4.886556,4.310368,386.002376
9,2023,2923.377472,861.095411,449.891647,38.973402,1573.417013,0.021643,0.347,511.922846,121.619274,5.064423,4.386026,390.981969


In [78]:
combined_base

,year,taxrev,amt,pymtc,ftc,taxbc,gbc_adj,tau,taxinc,sec199,taxDep,nid,ebitda
0,2014,320.731,4.457941,3.700228,101.582376,421.555664,0.021643,0.347,1295.669568,34.306208,884.575577,231.441998,2445.993351
1,2015,343.797,4.529305,3.799722,91.794692,434.862110,0.021643,0.347,1336.567506,34.855388,905.241147,243.981131,2520.645171
2,2016,299.571,4.524316,3.895522,96.318143,395.260349,0.021643,0.347,1214.849781,34.816996,830.188301,248.001094,2327.856171
3,2017,310.006,4.541886,3.978088,99.935946,409.378148,0.021643,0.347,1258.241448,34.952204,866.909916,268.261581,2428.365149
4,2018,324.343,4.550562,4.052119,96.600049,420.444606,0.021643,0.347,1292.254685,35.018974,820.971756,295.578104,2443.823519
5,2019,343.899,4.593510,4.117569,97.986091,441.409149,0.021643,0.347,1356.690115,35.349483,799.265383,328.930336,2520.235317
6,2020,380.350,4.659451,4.180064,103.553749,483.424362,0.021643,0.347,1485.825688,35.856932,635.671702,362.442904,2519.797226
7,2021,382.662,4.755976,4.243011,109.003947,491.152982,0.021643,0.347,1509.579935,36.599743,716.449543,393.691242,2656.320464
8,2022,389.490,4.886556,4.310368,114.806528,503.720339,0.021643,0.347,1548.206251,37.604625,785.544298,422.511919,2793.867094
9,2023,395.128,5.064423,4.386026,121.619274,516.068877,0.021643,0.347,1586.160018,38.973402,843.980733,448.477548,2917.591701


In [79]:
compare = np.asarray(combined_ref['taxrev'] - combined_base['taxrev'])

print "Total corporate tax revenue change, 2014-2027: " + str(np.round(sum(compare), 3))
print "Total corporate tax revenue change, 2017-2026: " + str(np.round(sum(compare[3:13]), 3))
print "Total corporate tax revenue change, 2018-2027: " + str(np.round(sum(compare[4:]), 3))

Total corporate tax revenue change, 2014-2027: -38.594
Total corporate tax revenue change, 2017-2026: -31.922
Total corporate tax revenue change, 2018-2027: -38.594


In [80]:
for i in compare:
    print i

5.68434188608e-14
-1.13686837722e-13
-1.13686837722e-13
1.70530256582e-13
-1.3475654793
-1.89323618346
-2.0408353885
-2.81460810043
-3.48762399245
-4.14603099108
-4.75904988852
-5.40861743972
-6.02406111484
-6.67273917367


## Calculate percent changes in investment income and pass-through income

In [81]:
execfile('multipliers_for_taxcalc.py')

In [82]:
indiv_gfactors

,Investment,year,inc_aftertax_base,inc_aftertax_ref,equity,debt,SchC_pos,SchC_neg,e26270_pos,e26270_neg
0,1105.741653,2014,1416.393849,1416.393849,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,1092.837295,2015,1386.540753,1386.540753,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,915.439816,2016,1179.897668,1179.897668,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,949.148567,2017,1213.679328,1213.679328,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
4,988.844086,2018,1151.313026,1152.660592,1.001170,1.000000,0.999656,1.001220,0.999509,1.001447
5,1024.661477,2019,1138.215655,1140.636158,1.002127,1.000765,0.999646,1.001944,0.999423,1.002257
6,1059.997174,2020,1031.411151,1034.508374,1.003003,1.001449,0.999744,1.002298,0.999491,1.002627
7,1099.923709,2021,1098.173702,1102.706229,1.004127,1.002069,0.999695,1.003195,0.999347,1.003642
8,1143.394137,2022,1163.520433,1169.496718,1.005136,1.002635,0.999685,1.003971,0.999258,1.004516
9,1188.796255,2023,1218.128548,1225.542174,1.006086,1.003153,0.999682,1.004665,0.999185,1.005295


## Pass changes to Tax-Calculator

In [83]:
indiv_rev_impact = distribute_results(iit_params_ref)

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.
You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [84]:
for x in indiv_rev_impact:
    print x
print sum(indiv_rev_impact[3:])

2.35991903467e-14
7.53214734706e-14
0.0
-6.73477593809e-14
0.14552016337
0.411305947956
0.753761191181
1.0889191491
1.43711081335
1.79627660981
2.14457786157
2.49816650786
2.86453693005
3.22623822988
16.3664134041


In [85]:
ModelResults = pd.DataFrame({'year': range(2014,2028), 'IndivTaxRev': indiv_rev_impact})
ModelResults['CorpTaxRev'] = combined_ref['taxrev'] - combined_base['taxrev']
ModelResults['RevenueChange'] = ModelResults['IndivTaxRev'] + ModelResults['CorpTaxRev']
ModelResults

,IndivTaxRev,year,CorpTaxRev,RevenueChange
0,2.359919e-14,2014,5.684342e-14,8.044261e-14
1,7.532147e-14,2015,-1.136868e-13,-3.836536e-14
2,0.000000e+00,2016,-1.136868e-13,-1.136868e-13
3,-6.734776e-14,2017,1.705303e-13,1.031825e-13
4,1.455202e-01,2018,-1.347565e+00,-1.202045e+00
5,4.113059e-01,2019,-1.893236e+00,-1.481930e+00
6,7.537612e-01,2020,-2.040835e+00,-1.287074e+00
7,1.088919e+00,2021,-2.814608e+00,-1.725689e+00
8,1.437111e+00,2022,-3.487624e+00,-2.050513e+00
9,1.796277e+00,2023,-4.146031e+00,-2.349754e+00


In [86]:
print "Total tax revenue change, 2014-2027: " + str(np.round(sum(ModelResults['RevenueChange']), 3))
print "Total tax revenue change, 2017-2026: " + str(np.round(sum(ModelResults['RevenueChange'][3:13]), 3))
print "Total tax revenue change, 2018-2027: " + str(np.round(sum(ModelResults['RevenueChange'][4:]), 3))

Total tax revenue change, 2014-2027: -22.228
Total tax revenue change, 2017-2026: -18.781
Total tax revenue change, 2018-2027: -22.228


In [87]:
for i in range(3,14):
    print ModelResults['RevenueChange'][i]
print '\n'
print sum(ModelResults['RevenueChange'][3:13])

1.03182497201e-13
-1.20204531593
-1.48193023551
-1.28707419731
-1.72568895133
-2.05051317909
-2.34975438127
-2.61447202695
-2.91045093186
-3.15952418479
-3.44650094379


-18.781453404
